In [40]:
!python -V

Python 3.12.11


# Homework
Basis: https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/01-intro

In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error 

In [42]:
%%bash
if [ ! -f "../data/yellow_tripdata_2023-01.parquet" ]; then
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -P ../data/
fi
if [ ! -f "../data/yellow_tripdata_2023-02.parquet" ]; then
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -P ../data/
fi

In [43]:
df_jan = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")

In [44]:
df_jan["duration"] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

# this way I can view methods of the single series or time_delta obj, since vsc does not index `td.` in lambda
# dir(df_jan["duration"].iloc[0])

df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_jan["duration"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [45]:
df_jan.duration.std()

np.float64(42.59435124195457)

In [46]:
a = len(df_jan)
df_jan_ft = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
b = len(df_jan_ft)
print(f"Data remaining after filter: {(b / a) * 100:.2f}%")

Data remaining after filter: 98.12%


In [47]:
dv = DictVectorizer()

categories = ['PULocationID', 'DOLocationID']
target = 'duration'

df_jan_ft[categories] = df_jan_ft[categories].astype(str)
df_jan_train = df_jan_ft[categories].to_dict(orient='records')

x_train = dv.fit_transform(df_jan_train)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_21884\591911440.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_ft[categories] = df_jan_ft[categories].astype(str)


In [48]:
y_train = df_jan_ft[target].values

In [49]:
# Q4. One-hot encoding
# I couldn't figure out other ways to count number of dimensions
x_train.ndim

2

In [50]:
#Q5. Training a model

lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_train)

root_mean_squared_error(y_train, y_pred)

7.64926193996255

In [51]:
# Q6. Evaluating the model
# For this one we will make a proper function going though everything we had before
def prepare_df_eval(df, target, dv, lr):
    
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categories = ['PULocationID', 'DOLocationID']

    df[categories] = df[categories].astype(str)
    df_val_dict = df[categories].to_dict(orient='records')

    # The number of features can be different between our sources
    # so we use `transform`` instead of `fit_transform`, to use same feature mapping of the original train dataset
    x_val = dv.transform(df_val_dict)
    y_val = df[target].values

    def eval_on_two_dfs(lr, x_val, y_val):
        y_pred = lr.predict(x_val)

        rmse = root_mean_squared_error(y_val, y_pred)
        return rmse
    return eval_on_two_dfs(lr, x_val, y_val)

prepare_df_eval(df_feb, target, dv, lr)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_21884\1340103467.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categories] = df[categories].astype(str)


7.811818059155752